In [46]:
import pandas as pd
import numpy as np

## Format into what ART takes

In [47]:
df = pd.read_csv('clean.csv')
df["Subject"] = df["subject_id"].apply(lambda x: x[4:])

In [17]:
# df["density"].replace({"low": 1, "medium": 2, "high": 3}, inplace=True)
# df["technique"].replace({"ConeExpand": 1, "2DMiniMap+": 2, "3DMiniMap": 3, "GravityZone+": 4}, inplace=True)
df["technique"].replace({"ConeExpand": "Cone", "2DMiniMap+": 'Two', "3DMiniMap": "Three", "GravityZone+": "Gravity"}, inplace=True)

In [18]:
searchdf = df[df["trial_type"] == "Search"].groupby(["Subject", "technique", "density"]).mean().reset_index()[["Subject", "technique", "density", "trial_time"]]
repeatdf = df[df["trial_type"] == "Repeat"].groupby(["Subject", "technique", "density"]).mean().reset_index()[["Subject", "technique", "density", "trial_time"]]

In [19]:
searchdf.Subject.unique()

array(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '21', '22', '23', '24', '25', '3', '4', '5', '6', '7', '8',
       '9'], dtype=object)

In [20]:
searchdf.to_csv("search.csv", index=False)
repeatdf.to_csv("repeat.csv", index=False)

## Process using ART and re-open

In [48]:
searchdf = pd.read_csv("search.art.csv")
repeatdf = pd.read_csv("repeat.art.csv")

In [22]:
searchdf.columns

Index(['Subject', 'technique', 'density', 'trial_time',
       'aligned(trial_time) for technique', 'aligned(trial_time) for density',
       'aligned(trial_time) for technique*density',
       'ART(trial_time) for technique', 'ART(trial_time) for density',
       'ART(trial_time) for technique*density'],
      dtype='object')

In [49]:
# reogranize into what SPSS takes

search_technique_spss_lines = []
search_density_spss_lines = []
search_interaction_spss_lines = []

for sub_idx, s in enumerate(searchdf.Subject.unique(), start=1):
    subject_row_technique = {"subjedct_id": sub_idx}
    subject_row_density = {"subjedct_id": sub_idx}
    subject_row_interaction = {"subjedct_id": sub_idx}

    for technique in searchdf.technique.unique():
        for density in searchdf.density.unique():

            pdf = searchdf[
                (searchdf["technique"] == technique) &
                (searchdf["density"] == density) &
                (searchdf["Subject"] == s)
            ]

            subject_row_technique["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique"].mean()
            subject_row_density["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for density"].mean()
            subject_row_interaction["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique*density"].mean()

    search_technique_spss_lines.append(subject_row_technique)
    search_density_spss_lines.append(subject_row_density)
    search_interaction_spss_lines.append(subject_row_interaction)

search_technique_spss = pd.DataFrame(search_technique_spss_lines)
search_density_spss = pd.DataFrame(search_density_spss_lines)
search_interaction_spss = pd.DataFrame(search_interaction_spss_lines)

search_technique_spss.to_csv("search_technique_spss.csv", index=False)
search_density_spss.to_csv("search_density_spss.csv", index=False)
search_interaction_spss.to_csv("search_interaction_spss.csv", index=False)

In [50]:
# reogranize into what SPSS takes

repeat_technique_spss_lines = []
repeat_density_spss_lines = []
repeat_interaction_spss_lines = []

for sub_idx, s in enumerate(repeatdf.Subject.unique(), start=1):
    subject_row_technique = {"subjedct_id": sub_idx}
    subject_row_density = {"subjedct_id": sub_idx}
    subject_row_interaction = {"subjedct_id": sub_idx}

    for technique in repeatdf.technique.unique():
        for density in repeatdf.density.unique():

            pdf = repeatdf[
                (repeatdf["technique"] == technique) &
                (repeatdf["density"] == density) &
                (repeatdf["Subject"] == s)
            ]

            subject_row_technique["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique"].mean()
            subject_row_density["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for density"].mean()
            subject_row_interaction["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique*density"].mean()

    repeat_technique_spss_lines.append(subject_row_technique)
    repeat_density_spss_lines.append(subject_row_density)
    repeat_interaction_spss_lines.append(subject_row_interaction)

repeat_technique_spss = pd.DataFrame(repeat_technique_spss_lines)
repeat_density_spss = pd.DataFrame(repeat_density_spss_lines)
repeat_interaction_spss = pd.DataFrame(repeat_interaction_spss_lines)

repeat_technique_spss.to_csv("repeat_technique_spss.csv", index=False)
repeat_density_spss.to_csv("repeat_density_spss.csv", index=False)
repeat_interaction_spss.to_csv("repeat_interaction_spss.csv", index=False)

## Contrasts testing

In [59]:
# Re-ogranize Search and Repeat for technique into what SPSS takes
# This requires to first run the ART-C with the technique condition

srch = "search-technique.art-c.csv"
rept = "repeat-technique.art-c.csv"

for f in [srch, rept]:
    df = pd.read_csv(f)
    df_technique_contrast = df.groupby(["Subject", "technique", "density"]).mean().reset_index().groupby(
        ["Subject", "technique"]).mean().reset_index()[["Subject", "technique", "ART-C(trial_time) for technique"]]

    rows = []
    for sub_idx, s in enumerate(df_technique_contrast.Subject.unique(), start=1):
        subject_row = {"subjedct_id": sub_idx}
        for technique in df_technique_contrast.technique.unique():
            pdf = df_technique_contrast[
                (df_technique_contrast["technique"] == technique) &
                (df_technique_contrast["Subject"] == s)
            ]
            subject_row[technique] = pdf["ART-C(trial_time) for technique"].mean()
        rows.append(subject_row)
    df_technique_contrast_spss = pd.DataFrame(rows)
    df_technique_contrast_spss.to_csv(f"contrast_{f[:6]}_technique_spss.csv", index=False)

In [60]:
# density contrast

srch = "search-density.art-c.csv"
rept = "repeat-density.art-c.csv"

for f in [srch, rept]:
    df = pd.read_csv(f)
    df_density_contrast = df.groupby(["Subject", "technique", "density"]).mean().reset_index().groupby(
        ["Subject", "density"]).mean().reset_index()[["Subject", "density", "ART-C(trial_time) for density"]]

    rows = []
    for sub_idx, s in enumerate(df_density_contrast.Subject.unique(), start=1):
        subject_row = {"subjedct_id": sub_idx}
        for density in df_density_contrast.density.unique():
            pdf = df_density_contrast[
                (df_density_contrast["density"] == density) &
                (df_density_contrast["Subject"] == s)
            ]
            subject_row[density] = pdf["ART-C(trial_time) for density"].mean()
        rows.append(subject_row)
    df_density_contrast_spss = pd.DataFrame(rows)
    df_density_contrast_spss.to_csv(f"contrast_{f[:6]}_density_spss.csv", index=False)

In [64]:
# interaction contrast

srch = "search-techniquedensity.art-c.csv"
rept = "repeat-techniquedensity.art-c.csv"

for f in [srch, rept]:
    df = pd.read_csv(f)
    df_interaction_contrast = df.groupby(["Subject", "techniquedensity"]).mean().reset_index()[
        ["Subject", "techniquedensity", "ART-C(trial_time) for techniquedensity"]]

    rows = []
    for sub_idx, s in enumerate(df_interaction_contrast.Subject.unique(), start=1):
        subject_row = {"subjedct_id": sub_idx}
        for techniquedensity in df_interaction_contrast.techniquedensity.unique():
            pdf = df_interaction_contrast[
                (df_interaction_contrast["techniquedensity"] == techniquedensity) &
                (df_interaction_contrast["Subject"] == s)
            ]
            subject_row[techniquedensity] = pdf["ART-C(trial_time) for techniquedensity"].mean()
        rows.append(subject_row)

    df_interaction_contrast_spss = pd.DataFrame(rows)
    df_interaction_contrast_spss.to_csv(
        f"contrast_{f[:6]}_interaction_spss.csv", index=False)

### OLD

In [34]:
df = pd.read_csv("search-density.art-c.csv")

In [42]:
seach_density_contrast_df = df[df["technique"]=="Two"].groupby(["Subject", "technique", "density"]).mean().reset_index().groupby(["Subject", "density"]).mean().reset_index()

In [43]:
seach_density_contrast_df.columns

Index(['Subject', 'density', 'trial_time', 'aligned(trial_time) for density',
       'ART-C(trial_time) for density'],
      dtype='object')

In [45]:
subject_rows = []

for sub_id, s in enumerate(seach_density_contrast_df.Subject.unique(), start=1):
    subject_row = {"subjedct_id": sub_id}
    for density in seach_density_contrast_df.density.unique():
        pdf = seach_density_contrast_df[
            (seach_density_contrast_df["Subject"] == s) &
            (seach_density_contrast_df["density"] == density)
        ]

        subject_row[str(density)] = pdf["ART-C(trial_time) for density"].mean()
        
        
    subject_rows.append(subject_row)

final_df = pd.DataFrame(subject_rows)
final_df.to_csv("seach_density_contrast_df_TWOD.csv", index=False)

        